In [ ]:
import pandas as pd
import numpy as np
import configparser

In [ ]:
!pip install --upgrade virtualenv

In [ ]:
!pip install haversine

In [ ]:
config = configparser.ConfigParser()
config.read('ais_config.ini')

In [ ]:
# read values from a section
ais_dataset_name = config.get('INIT', 'ais_dataset_name')
rotte_dataset_name = config.get('INIT', 'rotte_dataset_name')
#rottesort_filename = config.get('PREPROCESS', 'rottesort_filename')

world_porti_filename = config.get('INIT', 'world_porti_filename')
arrivi_nei_porti_fname = config.get('ELABORATION','arrivi_nei_porti_fname')
rotte_arrivi_porti_world_fname = config.get('ELABORATION','rotte_arrivi_porti_world_fname')
rotte_arrivi_porti_all_fname = config.get('ELABORATION','rotte_arrivi_porti_all_fname')

In [4]:
ais_dataset = pd.read_csv(ais_dataset_name, sep=',')
ais_dataset.head()

,lat,lng,speed,course,heading,elapsed,destination,flag,length,rot,...,w_left,dwt,gt_shiptype,shiptype,time,imo,mmsi,porto,type_summary,cod_porto
0,39.86600,8.547238,1.0,337.0,NaN,6,CASABLANCA,--,92.0,NaN,...,10.0,3658.0,9.0,7,2021-09-01,8820949,1193046,Oristano,Cargo,ITQOS
1,39.86599,8.547255,0.0,0.0,NaN,94,CASABLANCA,--,92.0,NaN,...,10.0,3658.0,9.0,7,2021-09-02,8820949,1193046,Oristano,Cargo,ITQOS
2,39.86369,8.552063,0.0,21.0,NaN,9,CASABLANCA,--,92.0,NaN,...,20.0,3658.0,9.0,7,2021-09-26,8407979,1193046,Oristano,Cargo,ITQOS
3,39.86369,8.552067,0.0,177.0,NaN,14,CASABLANCA,--,92.0,NaN,...,10.0,3658.0,9.0,7,2021-09-28,8407979,1193046,Oristano,Cargo,ITQOS
4,39.86602,8.547235,0.0,9.0,NaN,26,CASABLANCA,--,92.0,NaN,...,20.0,3658.0,9.0,7,2021-09-29,8407979,1193046,Oristano,Cargo,ITQOS


In [7]:
vessels_type = ais_dataset['type_summary'].unique()
vessels_flag = ais_dataset['flag'].unique()

In [8]:
print(vessels_type)
print(vessels_flag)

['Cargo' 'Tug' 'Pleasure Craft' nan 'Passenger' 'Tanker' 'Special Craft'
 'High Speed Craft' 'Fishing' 'Sailing Vessel' 'Search and Rescue' 'Other'
 'Unspecified']
['--' 'KY' 'IT' 'GB' 'DZ' 'MH' 'AL' 'BE' 'BG' 'CY' 'DE' 'MD' 'MT' 'DK'
 'ES' 'FR' 'GI' 'GR' 'HR' 'MA' 'NL' 'PA' 'KI' 'RU' 'BS' 'LU' 'MC' 'PT'
 'NO' 'SE' 'CH' 'TR' 'UA' 'LV' 'LT' 'AG' 'BM' 'BZ' 'BB' 'HN' 'US' 'JM'
 'KN' 'VC' 'VG' 'SA' 'CN' 'TW' 'AZ' 'IL' 'JP' 'KZ' 'KR' 'KW' 'LB' 'QA'
 'HK' 'PW' 'CK' 'NU' 'PH' 'SG' 'TV' 'VN' 'VU' 'WF' 'CM' 'KM' 'EG' 'LR'
 'PL' 'LY' 'SL' 'TG' 'TN']


In [5]:
vessels_mmsi = ais_dataset['mmsi'].unique().astype(int)
vessels_imo = ais_dataset['imo'].unique().astype(int)

print(len(vessels_mmsi))
print(len(vessels_imo))

3192
3164


In [6]:
rotte_dataset = pd.read_csv(rotte_dataset_name, sep=',', low_memory=False)

In [7]:
#drop NA values
rotte_dataset=rotte_dataset.dropna()
#Get index of rows containing wrong values
i=rotte_dataset[rotte_dataset.stamp=='stamp'].index
#drop rows containing wrong values
rotte_dataset.drop(i,inplace=True)

In [8]:
#build timestamp column to be added to rotte_092021 dataframe
stamp=rotte_dataset['stamp'].astype(float)
#stamp
from datetime import datetime
timestamp_column = stamp.apply(lambda x: datetime.fromtimestamp(x))
#timestamp_column

In [9]:
#add timestamp column to rotte_092021 dataframe
rotte_dataset['timestamp'] = timestamp_column.values

In [10]:
#sort_dataset by mmsi,stamp,lng,lat
rotte_dataset=rotte_dataset.sort_values(by=['mmsi','stamp','lng','lat'],ascending=[True,True,True,True])
#rotte_dataset.head()

In [37]:
#Analysis of data
#rotte_dataset_218158000 = rotte_dataset[rotte_dataset['mmsi']=='218158000']

rotte_dataset_201100115 = rotte_dataset[rotte_dataset['mmsi']=='201100115']


In [38]:
#Analysis of data
#rotte_dataset_218158000.to_csv("DCME\\AIS\\rotte_with_speed_2021\\rotte_arrivi_porti_set2021_218158000.csv",index=False,sep=',')
rotte_dataset_201100115.to_csv("DCME\\AIS\\rotte_with_speed_2021\\rotte_arrivi_porti_set2021_201100115.csv",index=False,sep=',')

In [11]:
class Utils(object):
   def intersect(self, nums1, nums2):
      """
      :type nums1: List[int]
      :type nums2: List[int]
      :rtype: List[int]
      """
      m = {}
      print(len(nums1))
      print(len(nums2))
      if len(nums1)<len(nums2):
         nums1,nums2 = nums2,nums1
      for i in nums1:
         if i not in m:
            m[i] = 1
         else:
            m[i]+=1
      result = []
      for i in nums2:
         if i in m and m[i]:
            m[i]-=1
            result.append(i)
      return result

In [12]:
rotte_dataset_mmsi =  rotte_dataset['mmsi'].unique().astype(int)
print(len(rotte_dataset_mmsi))

2840


In [13]:
utils = Utils()
mmsi_intersection = utils.intersect(vessels_mmsi.tolist(), rotte_dataset_mmsi.tolist())
print(f'result intersection: {mmsi_intersection}')

3192
2840
result intersection: [1193046, 201100115, 201100123, 201100124, 205481000, 205584000, 205688000, 207124000, 207137000, 207138000, 209016000, 209077000, 209092000, 209156000, 209240000, 209251000, 209262000, 209306000, 209410000, 209481000, 209510000, 209672000, 209681000, 209698000, 209714000, 209778000, 209881000, 209910000, 209950000, 209979000, 210121000, 210131000, 210652000, 211211850, 211215130, 211281610, 211307550, 211331640, 211362460, 211367460, 211382280, 211387390, 211428870, 211839000, 212028000, 212175000, 212184000, 212224000, 212267000, 212585000, 212613000, 212773000, 212853000, 212954000, 214182605, 214182719, 214182768, 214182779, 214182781, 214182784, 214182786, 214182788, 214182795, 214182798, 214182804, 215020000, 215034000, 215036000, 215054000, 215057000, 215063000, 215068000, 215072000, 215086000, 215105000, 215110000, 215135000, 215170000, 215179000, 215186000, 215200000, 215232000, 215234000, 215237000, 215238000, 215270000, 215311000, 215325000, 21

In [14]:
#Drop rows whose mmsi is not in rotte_092021_simplified(rotte_092021_mmsi,vessels_mmsi) from rotte_092021,
# i.e. get only rows whos mmsi is in mmsi_intersection

rotte_dataset_simplified = rotte_dataset[rotte_dataset['mmsi'].astype(int).isin(mmsi_intersection)]
#rotte_dataset_simplified.head()
#len(rotte_dataset_simplified)

In [15]:
rotte_dataset_simplified = rotte_dataset_simplified.reset_index()
#rotte_dataset_simplified.head()

ELABORAZIONE DELLE ROTTE

In [16]:
from datetime import datetime
import haversine as hs
from haversine import Unit

In [17]:
#elaborazione degli arrivi (df)
def arrival_elaboration(df_rotte):
    #df_rotte = rottesort

    dim=len(df_rotte)
    print("df_rotte len before: ",dim)

    df_arrival = pd.DataFrame(columns = ['row','mmsi','arrival','departure','lng','lat','lng_orig','lat_orig','speed','status'])

    oldmmsi=0
    sumrec=0
    start=0
    lat_orig=0
    lng_orig=0
    oldlng=0
    oldlat=0
    start='unknown'
    i=0
    status=0 ###0-nuovo 1-arrivato 2-partito

    start_time = datetime.now()

    for item in df_rotte.itertuples():
        try:
            #item = df_rotte.iloc[i]
            mmsi,time_voyage,lng,lat,speed = item.mmsi,item.timestamp,float(item.lng),float(item.lat),int(item.speed)

            if(mmsi!=oldmmsi):
                if (status==1):#si riferisce alla old ship
                    df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,'lng_orig':lng_orig,'lat_orig':lat_orig,
                            'speed':speed,'status':status},ignore_index=True)
                start,status,lng_orig,lat_orig,oldlng,oldlat,oldmmsi='unknown',0,0,0,0,0,mmsi
                oldmmsi = mmsi
    
            if(speed==0):          
                if (status==0):
                    start,oldlng,oldlat = time_voyage,lng,lat
                
                #To calculate distance in meters
                if(status<2):
                    loc1=(lat,lng)
                    loc2=(oldlat,oldlng)
            
                    distance = hs.haversine(loc1,loc2,unit=Unit.METERS)
            
                    if(distance > 3000.0):
                        #print(distance)
                        #if ((abs(oldlng-lng)+abs(oldlat-lat))>0.3):
                        df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,
                                       'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':status},ignore_index=True)
                        start,lng_orig,lat_orig,oldlng,oldlat = time_voyage,oldlng,oldlat,lng,lat
                    
                if(status==2):
                    start,lng_orig,lat_orig,oldlng,oldlat = time_voyage,oldlng,oldlat,lng,lat
                    
                end=time_voyage
                status=1
                
            if (speed>0):
                if(status==1):
                    loc1=(lat,lng)
                    loc2=(oldlat,oldlng)
                    distance = hs.haversine(loc1,loc2,unit=Unit.METERS)
                    if(distance > 3000.0):
                        df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                           'departure':end,'lng':oldlng,'lat':oldlat,
                            'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':2},ignore_index=True)
                        
                        status,lng_orig,lat_orig = 2,oldlng,oldlat
                if(status==0):
                    status=2
                   
            #stampa di controllo
            if(i%1000000 == 0):
               print(i)
        except Exception as e:
            print(i," ",speed)
            print(e)
        i+=1
    #end for
    
    #scrive l'ultimo record
    if(status==1):
            df_arrival=df_arrival.append({'row':i,'mmsi':oldmmsi,'arrival':start,
                        'departure':end,'lng':oldlng,'lat':oldlat,
                        'lng_orig':lng_orig,'lat_orig':lat_orig,'speed':speed,'status':status},ignore_index=True)

 
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("df_arrival len after: ",len(df_arrival))
    
    return(df_arrival)

In [18]:
df_rotte = rotte_dataset_simplified[['mmsi','stamp','timestamp','lng','lat','speed']]
df_rotte = df_rotte.sort_values(by=['mmsi','stamp'],ascending=[True,True])
df_rotte = df_rotte.reset_index()

In [19]:
df_arrival= arrival_elaboration(df_rotte)

df_rotte len before:  33474627
0
1000000
2000000
3000000
4000000
5000000
6000000
7000000
8000000
9000000
10000000
11000000
12000000
13000000
14000000
15000000
16000000
17000000
18000000
19000000
20000000
21000000
22000000
23000000
24000000
25000000
26000000
27000000
28000000
29000000
30000000
31000000
32000000
33000000
Duration: 0:09:52.562333
df_arrival len after:  83202


In [29]:
#print(len(df_arrival))
#6518
#df_arrival[(df_arrival['speed']==0) & (df_arrival['status'] == 2)] # 0 records
df_arrival[(df_arrival['speed']==0)]

,row,mmsi,arrival,departure,lng,lat,lng_orig,lat_orig,speed,status
0,1,1193046,2021-09-01 02:11:25,2021-09-01 02:11:25,8.398606,40.83946,0,0,0,1
1,2,1193046,2021-09-01 02:28:00,2021-09-01 02:28:00,23.557100,37.96413,8.398606,40.83946,0,1
2,3,1193046,2021-09-01 02:29:25,2021-09-01 02:29:25,8.398600,40.83945,23.5571,37.96413,0,1
3,6,1193046,2021-09-01 02:32:02,2021-09-01 02:40:40,23.557080,37.96413,8.3986,40.83945,0,1
4,7,1193046,2021-09-01 02:41:26,2021-09-01 02:41:26,8.398599,40.83947,23.55708,37.96413,0,1
...,...,...,...,...,...,...,...,...,...,...
83098,33368262,671500000,2021-10-05 08:20:08,2021-10-09 11:25:13,13.231230,45.79362,13.29773,45.64499,0,1
83104,33372383,671500000,2021-10-25 22:05:21,2021-10-26 10:59:40,34.638480,36.71656,33.01963,34.6552,0,1
83193,33467435,672748000,2021-09-21 00:03:09,2021-10-29 11:27:07,10.304250,36.80750,5.342587,43.33676,0,1
83194,33469384,69519104,2021-10-24 21:43:44,2021-11-01 00:57:02,10.036100,44.03262,0,0,0,1


In [20]:
world_porti = pd.read_csv(world_porti_filename, sep=',',low_memory=False)

In [21]:
len(world_porti)



13709

In [22]:
ita_porti = world_porti[world_porti['Country']=='ITA']
len(ita_porti)

457

In [23]:
world_porti=world_porti[['Country','Name','Latitude','Longitude']]

In [24]:
world_porti.reset_index(drop=True,inplace=True)

In [25]:
def port_assign(df_rotte,df_porti):
    
    n=len(df_rotte)
    df = df_rotte
    df["port"] = ["null"]*n
    df["port_orig"] = ["null"]*n
    
    start_time = datetime.now()
    
    i=0
    for rotta in df_rotte.itertuples():
        found_orig,found=0,0
        #departure port position
        loc_orig=(float(rotta.lat_orig),float(rotta.lng_orig))
        #arrival port position
        loc=(float(rotta.lat),float(rotta.lng))
        
        for porto in df_porti.itertuples():            
            loc_port=(float(porto.Latitude),float(porto.Longitude))            
            distance1 = hs.haversine(loc,loc_port,unit=Unit.METERS)
            if(distance1 < 5000.0):
                df.loc[i,'port']=porto.Name
                found=1
                break
        if (found==0):
            df.loc[i,'port']='Not found'
        
        for porto in df_porti.itertuples():             
            loc_port=(float(porto.Latitude),float(porto.Longitude))
            distance2 = hs.haversine(loc_orig,loc_port,unit=Unit.METERS)
            if(distance2 < 5000.0):
                df.loc[i,'port_orig']=porto.Name
                found_orig=1
                break    

        if (found_orig==0):
            df.loc[i,'port_orig']='Not found'
            
        #stampa di controllo
        if(i%10000 == 0):
               print(i)
        i+=1
    
    end_time = datetime.now()
    print('Duration: {}'.format(end_time - start_time))
    print("df_arrival len after: ",len(df_arrival))
        
    return(df)

In [26]:
df_rotte_arrivi_porti = port_assign(df_arrival,world_porti)

0
10000
20000
30000
40000
50000
60000
70000
80000
Duration: 11:21:22.024182
df_arrival len after:  83202


ADD INFORMATION TO THE DATASET

In [27]:
vessels_mmsi_imo_dict = pd.Series(ais_dataset.imo.values,index=ais_dataset.mmsi).to_dict()
#vessels_mmsi_callsign_dict = pd.Series(ais_dataset.callsign.values,index=ais_dataset.mmsi).to_dict()
vessels_mmsi_type_summary_dict = pd.Series(ais_dataset.type_summary.values,index=ais_dataset.mmsi).to_dict()

vessels_mmsi_imo_clean_dict = {k: vessels_mmsi_imo_dict[k] for k in vessels_mmsi_imo_dict if not pd.isna(vessels_mmsi_imo_dict[k])}
#vessels_mmsi_callsign_clean_dict = {k: vessels_mmsi_callsign_dict[k] for k in vessels_mmsi_callsign_dict if not pd.isna(vessels_mmsi_callsign_dict[k])}
vessels_mmsi_type_summary_clean_dict = {k: vessels_mmsi_type_summary_dict[k] for k in vessels_mmsi_type_summary_dict if not pd.isna(vessels_mmsi_type_summary_dict[k])}

In [28]:
df_rotte_arrivi_porti["imo"] = df_rotte_arrivi_porti["mmsi"].astype(int)
df_rotte_arrivi_porti['imo'].replace(vessels_mmsi_imo_clean_dict ,inplace=True)

#df_rotte_arrivi_porti["callsign"] = df_rotte_arrivi_porti["mmsi"].astype(int)
#df_rotte_arrivi_porti['callsign'].replace(vessels_mmsi_callsign_clean_dict ,inplace=True)

df_rotte_arrivi_porti['type_summary'] = df_rotte_arrivi_porti["mmsi"].astype(int)
df_rotte_arrivi_porti['type_summary'].replace(vessels_mmsi_type_summary_clean_dict ,inplace=True)

In [29]:
#export all records
df_rotte_arrivi_porti.to_csv(rotte_arrivi_porti_all_fname,index=False,sep=',')
df_rotte_arrivi_porti

,row,mmsi,arrival,departure,lng,lat,lng_orig,lat_orig,speed,status,port,port_orig,imo,type_summary
0,1,1193046,2021-09-01 02:11:25,2021-09-01 02:11:25,8.398606,40.83946,0,0,0,1,Porto Torres,Not found,7512507,Tug
1,2,1193046,2021-09-01 02:28:00,2021-09-01 02:28:00,23.557100,37.96413,8.398606,40.83946,0,1,Keratsini (Agios Georgios),Porto Torres,7512507,Tug
2,3,1193046,2021-09-01 02:29:25,2021-09-01 02:29:25,8.398600,40.83945,23.5571,37.96413,0,1,Porto Torres,Keratsini (Agios Georgios),7512507,Tug
3,6,1193046,2021-09-01 02:32:02,2021-09-01 02:40:40,23.557080,37.96413,8.3986,40.83945,0,1,Keratsini (Agios Georgios),Porto Torres,7512507,Tug
4,7,1193046,2021-09-01 02:41:26,2021-09-01 02:41:26,8.398599,40.83947,23.55708,37.96413,0,1,Porto Torres,Keratsini (Agios Georgios),7512507,Tug
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83197,33474188,777777777,2021-10-08 22:34:48,2021-10-09 11:10:57,37.568370,55.71177,0,0,11,2,Not found,Not found,9806275,Pleasure Craft
83198,33474222,777777777,2021-10-11 21:58:48,2021-10-11 23:45:40,33.643410,34.92853,37.56837,55.71177,77,2,Larnaca Fishing Shelter,Not found,9806275,Pleasure Craft
83199,33474361,777777777,2021-10-13 21:55:32,2021-10-14 06:19:41,37.377210,56.88423,33.64341,34.92853,108,2,Not found,Larnaca Fishing Shelter,9806275,Pleasure Craft
83200,33474398,777777777,2021-10-14 22:22:42,2021-10-14 23:02:52,-17.433410,14.68837,37.37721,56.88423,145,2,Dakar,Not found,9806275,Pleasure Craft


In [46]:
#export only records such that a port has been found
df_rotte_arrivi_porti_world_found = df_rotte_arrivi_porti[(df_rotte_arrivi_porti['port'] != 'Not found')]
df_rotte_arrivi_porti_world_found.to_csv(rotte_arrivi_porti_world_fname,index=False,sep=',')

COMPUTE STATISTICS OF ARRIVALS IN THE PORTS

In [ ]:
df_rotte_arrivi_porti_world_found.query("status > 0").groupby(['port']).count().to_csv(arrivi_nei_porti_fname,sep=';')

In [47]:
#check
df_rotte_arrivi_porti_world_found[(df_rotte_arrivi_porti_world_found['speed']==0)]

,row,mmsi,arrival,departure,lng,lat,lng_orig,lat_orig,speed,status,port,port_orig,imo,type_summary
0,1,1193046,2021-09-01 02:11:25,2021-09-01 02:11:25,8.398606,40.83946,0,0,0,1,Porto Torres,Not found,7512507,Tug
1,2,1193046,2021-09-01 02:28:00,2021-09-01 02:28:00,23.557100,37.96413,8.398606,40.83946,0,1,Paloukia,Porto Torres,7512507,Tug
2,3,1193046,2021-09-01 02:29:25,2021-09-01 02:29:25,8.398600,40.83945,23.5571,37.96413,0,1,Porto Torres,Akti Tzelepi,7512507,Tug
3,6,1193046,2021-09-01 02:32:02,2021-09-01 02:40:40,23.557080,37.96413,8.3986,40.83945,0,1,Paloukia,Porto Torres,7512507,Tug
4,7,1193046,2021-09-01 02:41:26,2021-09-01 02:41:26,8.398599,40.83947,23.55708,37.96413,0,1,Porto Torres,Akti Tzelepi,7512507,Tug
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83051,33295062,671105100,2021-10-11 21:34:01,2021-10-12 11:25:02,33.314930,34.71889,14.41679,42.35447,0,1,Vasilikos,Ortona,8913021,Cargo
83096,33367033,671500000,2021-10-02 10:28:48,2021-10-04 11:27:39,12.263500,45.21023,12.37312,45.24751,0,1,Chioggia,Chioggia,8917730,Cargo
83098,33368262,671500000,2021-10-05 08:20:08,2021-10-09 11:25:13,13.231230,45.79362,13.29773,45.64499,0,1,Porto Nogaro,Grado,8917730,Cargo
83193,33467435,672748000,2021-09-21 00:03:09,2021-10-29 11:27:07,10.304250,36.80750,5.342587,43.33676,0,1,La Goulette Nord (Halqueloued),L'Estaque,9598579,Passenger


In [4]:
#df_rotte_all = pd.read_csv(rotte_arrivi_porti_all_fname, sep=',', low_memory=False)

In [31]:
#df_rotte_passengers_cargo = df_rotte_all[(df_rotte_all['type_summary']=='Passenger') | (df_rotte_all['type_summary']=='Cargo')]


df_rotte_arrivi_porti_pass_cargo = df_rotte_arrivi_porti[(df_rotte_arrivi_porti['type_summary']=='Passenger') | (df_rotte_arrivi_porti['type_summary']=='Cargo')]

In [32]:
len(df_rotte_arrivi_porti_pass_cargo)

40080

In [33]:
df_rotte_arrivi_porti_pass_cargo.to_csv("DCME\\AIS\\rotte_with_speed_2021\\rotte_arrivi_porti_passengers_cargo_set2021.csv",index=False,sep=',')